In [25]:
import pandas as pd
import numpy as np

# 1. File Reading: Read the file once and extract required sheets

xls = pd.ExcelFile("FantaSPL_V2.2.xlsx")
roles_df = xls.parse(sheet_name=3)
data_first_sheet = xls.parse(sheet_name=0)
punti_df = xls.parse(sheet_name="Punti")
parameters_df = xls.parse(sheet_name="Parametri")

# 2. Column Renaming and Data Transformation
# Provided dictionaries for translations
translation_dict = {
    "Difensore": "Defender",
    "Centrocampista": "Midfielder",
    "Attaccante": "Forward",
    "Portiere": "Goalkeeper"
}
column_mapping_first_sheet = {
    "Data": "Date",
    "Tipo Partita": "Match Type",
    "Bianchi": "Team A Goals",
    "Scuri": "Team B Goals",
    "Squadra vincente": "Winning Team",
    "Numero Goal": "Number of Goals",
    "Goal Check": "Goal Check",
    "Numero Goal Punti": "Goal Points",
    "Autogoal": "Own Goals",
    "Costo per Partita": "Cost per Match",
    "Effettivo": "Actual Cost"
}
translations = {
    "Data": "Date",
    "Player": "Player",
    "Ruolo": "Role",
    "Colore Squadra": "Team Color",
    "Vincitori": "Winners",
    "Vinta": "Won",
    "#Goal": "Goals",
    "Goal Subiti": "Goals Conceded",
    "Goal Fatti": "Goals Scored",
    "AutoGoal": "Own Goals",
    "SPL Bonus": "SPL Bonus",
    "MVP": "MVP",
    "Amici Portati": "Friends Brought",
    "Penalità": "Penalties",
    "Presenza": "Attendance",
    "Penalità.1": "Penalties.1",
    "Punteggio Difesa": "Defense Score",
    "Punteggio Centrocampisti": "Midfielders Score",
    "Squadra Vincente": "Winning Team",
    "#Goal.1": "Goals.1",
    "AutoGoal.1": "Own Goals.1",
    "SPL Bonus.1": "SPL Bonus.1",
    "MVP.1": "MVP.1",
    "Porta Amico": "Friend Referral",
    "TOTAL": "TOTAL"
}

# Merging the dictionaries for translations and column mapping
translations.update(translation_dict)
translations.update(column_mapping_first_sheet)

# Continue with the rest of the optimization
roles_df["Position"] = roles_df["Ruolo"].map(translations)
data_first_sheet.rename(columns=translations, inplace=True)
punti_df.rename(columns=translations, inplace=True)

# Transformations
user_input_table = data_first_sheet[["Date", "Match Type", "Team A Goals", "Team B Goals"]].copy()
user_input_table["Winning Team"] = user_input_table.apply(
    lambda row: "A" if row["Team A Goals"] > row["Team B Goals"] else ("B" if row["Team A Goals"] < row["Team B Goals"] else "X"), axis=1
)
user_input_table["Total Goals"] = user_input_table["Team A Goals"] + user_input_table["Team B Goals"]

# Further transformations for punti_df
user_input_df = punti_df[['Date', 'Player', 'Team Color', 'Goals', 'Own Goals', 'SPL Bonus', 'MVP', 'Friends Brought', 'Penalties']].copy()
user_input_df['Team Color'].replace({'Scuri': 'B', 'Bianchi': 'A'}, inplace=True)
replacement_dict = {'Sì': 1, 'No': 0}
for column in ['SPL Bonus', 'MVP', 'Penalties']:
    user_input_df[column].replace(replacement_dict, inplace=True)

# Efficient game_outcome function
def game_outcome(row):
    if row['Winning Team'] == 'X':
        return 'Draw'
    elif row['Team Color'] == row['Winning Team']:
        return 'Won'
    return 'Lost'

merged_table = user_input_df.merge(user_input_table, on='Date', how='inner')
merged_table['Game Outcome'] = merged_table.apply(game_outcome, axis=1)
merged_table['Goals Conceded'] = merged_table.apply(
    lambda row: row['Team B Goals'] if row['Team Color'] == 'A' else row['Team A Goals'],
    axis=1
)
merged_table['Goals Scored'] = merged_table.apply(
    lambda row: row['Team A Goals'] if row['Team Color'] == 'A' else row['Team B Goals'],
    axis=1
)
merged_table = merged_table.merge(roles_df[['Player', 'Position']], on='Player', how='left')
final_columns = ["Date", "Player", "Position", "Team Color", "Game Outcome", "Goals", "Own Goals", "Goals Conceded", "Goals Scored", "SPL Bonus", "MVP", "Friends Brought", "Penalties"]
merged_table = merged_table[final_columns]

def position_7(position_11):
    if position_11 == "Defender":
        return "Defensive"
    elif position_11 in ["Midfielder", "Forward"]:
        return "Offensive"
    elif position_11 == "Goalkeeper":
        return "Goalkeeper"

def position_5(position_11):
    if position_11 in ["Defender", "Midfielder", "Forward"]:
        return "Outfield"
    elif position_11 == "Goalkeeper":
        return "Goalkeeper"

roles_df['Position 7'] = roles_df['Position'].apply(position_7)
roles_df['Position 5'] = roles_df['Position'].apply(position_5)
merged_table = merged_table.merge(roles_df[['Player', 'Position 7', 'Position 5']], on='Player', how='left')

# Converting parameters_df to a dictionary for efficient lookup
parameters_dict = {
    '5-a-side': parameters_df.set_index('Parameter')['5-a-side'].to_dict(),
    '7-a-side': parameters_df.set_index('Parameter')['7-a-side'].to_dict(),
    '11-a-side': parameters_df.set_index('Parameter')['11-a-side'].to_dict()
}

# Function to get parameter value based on match type and parameter name
def get_parameter_value(match_type, parameter):
    if match_type <= 10:
        return parameters_dict['5-a-side'].get(parameter, 0)
    elif match_type <= 16:
        return parameters_dict['7-a-side'].get(parameter, 0)
    else:
        return parameters_dict['11-a-side'].get(parameter, 0)

# Adding new columns to the merged_table

# Checking if 'Match Type' column is present in user_input_table
if 'Match Type' in user_input_table.columns:
    # Merging 'Match Type' column to merged_table
    merged_table = merged_table.merge(user_input_table[['Date', 'Match Type']], on='Date', how='left')
else:
    raise ValueError("'Match Type' column is missing in user_input_table.")

# Re-applying the scoring calculations

# Participation points
merged_table['Participation Points'] = merged_table['Match Type'].apply(lambda x: get_parameter_value(x, 'Participation'))

def game_outcome_parameter_value(match_type, outcome):
    if outcome == "Won":
        return get_parameter_value(match_type, 'Win')
    elif outcome == "Lost":
        return get_parameter_value(match_type, 'Loss')
    else:  # outcome == "Draw"
        return get_parameter_value(match_type, 'Draw')

# Adjusting 'Game Outcome Points' column
merged_table['Game Outcome Points'] = merged_table.apply(
    lambda row: game_outcome_parameter_value(row['Match Type'], row['Game Outcome']),
    axis=1
)

# Goal points
merged_table['Goal Points'] = merged_table.apply(
    lambda row: row['Goals'] * get_parameter_value(row['Match Type'], 'Goal'),
    axis=1
)

# Own goal points
merged_table['Own Goal Points'] = merged_table.apply(
    lambda row: row['Own Goals'] * get_parameter_value(row['Match Type'], 'Own Goal'),
    axis=1
)

# Penalty points
merged_table['Penalty Points'] = merged_table.apply(
    lambda row: row['Penalties'] * get_parameter_value(row['Match Type'], 'Penalty'),
    axis=1
)

# SPL bonus points
merged_table['SPL Bonus Points'] = merged_table.apply(
    lambda row: row['SPL Bonus'] * get_parameter_value(row['Match Type'], 'SPL Bonus'),
    axis=1
)

# MVP points
merged_table['MVP Points'] = merged_table.apply(
    lambda row: row['MVP'] * get_parameter_value(row['Match Type'], 'MVP'),
    axis=1
)

# Friends brought points
merged_table['Friends Brought Points'] = merged_table.apply(
    lambda row: row['Friends Brought'] * get_parameter_value(row['Match Type'], 'Friend Brought'),
    axis=1
)


# Define functions to calculate Defensive and Midfield scores based on the match type and position

def defensive_score_11(position, goals_conceded):
    if position == "Goalkeeper":
        return get_parameter_value(17, 'Defence Score') - goals_conceded + get_parameter_value(17, 'Goalkeeper Score')
    elif position == "Defender":
        return get_parameter_value(17, 'Defence Score') - goals_conceded
    elif position == "Midfielder":
        return np.ceil((get_parameter_value(17, 'Defence Score') - goals_conceded) / 2)
    else:
        return 0  # Forwards and other positions

def midfield_score_11(position, goal_difference):
    if position in ["Midfielder", "Forward"]:
        score = max(0, goal_difference)
        return score if position == "Midfielder" else np.ceil(score / 3)
    else:
        return 0  # Goalkeepers, Defenders, and other positions

def defensive_score_7(position, goals_conceded):
    if position == "Goalkeeper":
        return get_parameter_value(13, 'Defence Score') - goals_conceded + get_parameter_value(13, 'Goalkeeper Score')
    elif position == "Defensive":
        return get_parameter_value(13, 'Defence Score') - goals_conceded
    else:
        return 0  # Offensive and other positions

def midfield_score_7(position, goal_difference):
    if position == "Offensive":
        return max(0, goal_difference)
    else:
        return 0  # Goalkeepers, Defensive, and other positions

def defensive_score_5(position, goals_conceded):
    if position == "Goalkeeper":
        return get_parameter_value(5, 'Defence Score') - goals_conceded + get_parameter_value(5, 'Goalkeeper Score')
    elif position == "Outfield":
        return get_parameter_value(5, 'Defence Score') - goals_conceded
    else:
        return 0  # Offensive and other positions

def midfield_score_5(position, goal_difference):
    if position == "Outfield":
        return max(0, goal_difference)
    else:
        return 0  # Goalkeepers and other positions

# Apply these functions to the merged table
merged_table['Defensive Score'] = merged_table.apply(
    lambda row: defensive_score_11(row['Position'], row['Goals Conceded']) if row['Match Type'] > 16
    else (defensive_score_7(row['Position 7'], row['Goals Conceded']) if row['Match Type'] > 10
          else defensive_score_5(row['Position 5'], row['Goals Conceded'])),
    axis=1
)

merged_table['Midfield Score'] = merged_table.apply(
    lambda row: midfield_score_11(row['Position'], row['Goals Scored'] - row['Goals Conceded']) if row['Match Type'] > 16
    else (midfield_score_7(row['Position 7'], row['Goals Scored'] - row['Goals Conceded']) if row['Match Type'] > 10
          else midfield_score_5(row['Position 5'], row['Goals Scored'] - row['Goals Conceded'])),
    axis=1
)

# Ensuring non-negative values for 'Defensive Score' and 'Midfield Score' columns
merged_table['Defensive Score'] = merged_table['Defensive Score'].apply(lambda x: max(0, x))
merged_table['Midfield Score'] = merged_table['Midfield Score'].apply(lambda x: max(0, x))

# Displaying the first few rows of the updated merged_table
merged_table.tail(20)



,Date,Player,Position,Team Color,Game Outcome,Goals,Own Goals,Goals Conceded,Goals Scored,SPL Bonus,MVP,Friends Brought,Penalties,Position 7,Position 5,Match Type,Participation Points,Game Outcome Points,Goal Points,Own Goal Points,Penalty Points,SPL Bonus Points,MVP Points,Friends Brought Points,Defensive Score,Midfield Score
401,2023-07-11,Daniele (Francesco),Midfielder,A,Won,2,0,3,4,0,0,0,0,Offensive,Outfield,14,4,3,2,0,0,0,0,0,0.0,1.0
402,2023-07-11,Fabrizio Limonta,Defender,A,Won,1,0,3,4,0,0,0,0,Defensive,Outfield,14,4,3,1,0,0,0,0,0,7.0,0.0
403,2023-07-11,Maurizio,Midfielder,A,Won,0,0,3,4,0,0,0,0,Offensive,Outfield,14,4,3,0,0,0,0,0,0,0.0,1.0
404,2023-07-11,Robi (Stoppi),Defender,A,Won,0,0,3,4,0,0,0,0,Defensive,Outfield,14,4,3,0,0,0,0,0,0,7.0,0.0
405,2023-07-11,Andrea DeGa,Midfielder,B,Lost,0,0,4,3,0,0,0,0,Offensive,Outfield,14,4,0,0,0,0,0,0,0,0.0,0.0
406,2023-07-11,Andrea Silverstri,Defender,B,Lost,0,0,4,3,0,0,0,0,Defensive,Outfield,14,4,0,0,0,0,0,0,0,6.0,0.0
407,2023-07-11,Luca Stoppi,Defender,B,Lost,1,0,4,3,0,0,0,0,Defensive,Outfield,14,4,0,1,0,0,0,0,0,6.0,0.0
408,2023-07-11,Emilano (Dani),Midfielder,B,Lost,1,0,4,3,0,0,0,0,Offensive,Outfield,14,4,0,1,0,0,0,0,0,0.0,0.0
409,2023-07-11,Sergio,Midfielder,B,Lost,1,0,4,3,0,0,0,0,Offensive,Outfield,14,4,0,1,0,0,0,0,0,0.0,0.0
410,2023-07-11,Pedro,Defender,B,Lost,0,0,4,3,0,0,0,0,Defensive,Outfield,14,4,0,0,0,0,0,0,0,6.0,0.0


In [ ]:


# Display up to 50 rows
pd.set_option('display.max_rows', 50)

# Display all columns
pd.set_option('display.max_columns', None)

# Increase the width of each column for better visibility
pd.set_option('display.max_colwidth', 100)

# Now, when you display a DataFrame, it will show up to 50 rows and all columns.
merged_table.head(50)


In [24]:
# Filtering and displaying rows where the 'Position' is "Defender"
defender_rows = merged_table[merged_table['Position'] == 'Defender']
defender_rows.head(50)  # Displaying first 50 rows for "Defender" position


,Date,Player,Position,Team Color,Game Outcome,Goals,Own Goals,Goals Conceded,Goals Scored,SPL Bonus,MVP,Friends Brought,Penalties,Position 7,Position 5,Match Type,Participation Points,Game Outcome Points,Goal Points,Own Goal Points,Penalty Points,SPL Bonus Points,MVP Points,Friends Brought Points,Defensive Score,Midfield Score
0,2023-01-12,Andrea Limonta,Defender,B,Lost,0,0,18,13,1,0,0,0,Defensive,Outfield,14,4,0,0,0,0,3,0,0,0.0,0.0
1,2023-01-12,Luca Stoppi,Defender,B,Lost,2,1,18,13,0,0,0,0,Defensive,Outfield,14,4,0,2,-2,0,0,0,0,0.0,0.0
7,2023-01-12,Fabrizio Limonta,Defender,A,Won,0,0,13,18,0,0,0,0,Defensive,Outfield,14,4,3,0,0,0,0,0,0,0.0,0.0
9,2023-01-12,Giovanni Aiello,Defender,A,Won,1,0,13,18,0,0,0,0,Defensive,Outfield,14,4,3,1,0,0,0,0,0,0.0,0.0
10,2023-01-12,Cri Diaz Lopez,Defender,A,Won,1,0,13,18,0,0,0,0,Defensive,Outfield,14,4,3,1,0,0,0,0,0,0.0,0.0
13,2023-01-12,Federico Paolucci,Defender,A,Won,2,0,13,18,0,0,0,0,Defensive,Outfield,14,4,3,2,0,0,0,0,0,0.0,0.0
14,2023-01-19,Andrea Limonta,Defender,A,Lost,0,0,7,4,0,0,0,0,Defensive,Outfield,22,5,0,0,0,0,0,0,0,0.0,0.0
15,2023-01-19,Luca Stoppi,Defender,A,Lost,0,0,7,4,0,0,2,0,Defensive,Outfield,22,5,0,0,0,0,0,0,0,0.0,0.0
18,2023-01-19,Fabrizio Limonta,Defender,A,Lost,0,0,7,4,0,0,0,0,Defensive,Outfield,22,5,0,0,0,0,0,0,0,0.0,0.0
20,2023-01-19,Giovanni Aiello,Defender,A,Lost,0,0,7,4,0,0,0,0,Defensive,Outfield,22,5,0,0,0,0,0,0,0,0.0,0.0
